In [34]:
import cv2
from imutils.video import VideoStream
import tensorflow as tf
import tensorflow.keras
import time
import imutils
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [35]:
prototxtpath = './face_detector/deploy.prototxt'
weightspath = './face_detector/res10_300x300_ssd_iter_140000.caffemodel'

In [43]:
def detect_and_predict(frame,faceNet,maskNet):
    (h,w)=frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    faces=[]
    locs = []
    preds=[]
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence >0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = tf.keras.preprocessing.image.img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            faces.append(face)
            locs.append((startX,startY,endX,endY))
    if(len(faces)>0):
        preds = maskNet.predict(faces)
    return (locs,preds)
            

            

In [39]:
faceNet = cv2.dnn.readNet(prototxtpath,weightspath)

In [40]:
maskNet = tf.keras.models.load_model('./my_mask_detector.h5')

In [41]:
vs = VideoStream(0,resolution=(400,400)).start()
time.sleep(2.0)

In [44]:
while True:
    frame = vs.read()
    frame = imutils.resize(frame,width=400)
    (locs, preds) = detect_and_predict(frame, faceNet, maskNet)
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        label = "Mask"if mask>withoutMask else "No mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) == 27:#press esc to exit
            break
cv2.destroyAllWindows()
vs.stop()
    
    